In [1]:
%pip install nbformat
%pip install requests
%pip install configparser
%pip install ipywidgets
%pip install pandas
%pip install matplotlib

In [2]:
%run ../src/helpers.ipynb

In [12]:
import unittest
from unittest.mock import MagicMock
from datetime import datetime, timedelta
from configparser import ConfigParser
import pandas as pd

In [19]:
class TestHelpers(unittest.TestCase):
    def test_get_rates_dataframe(self):
        """
        Compares the test datafram with the expected results returnd by `get_rates_dataframe` function.
        
        - Setup: Defines a sample input data dictionary containing exchange rate information.    
        - Expected Result: Constructs the expected output DataFrame based on the sample data provided.
        - Function Execution: Calls the `get_rates_dataframe` function with the sample data.
        - Assertion: Checks whether the generated DataFrame matches the expected DataFrame.
        """

        # Sample input data
        sample_data = {
            "success": True,
            "timeseries": True,
            "start_date": "2024-02-20",
            "end_date": "2024-03-21",
            "base": "AUD",
            "rates": {
                "2024-02-20": {"NZD": 1.062553, "GBP": 0.519046},
                "2024-02-21": {"NZD": 1.060413, "GBP": 0.518546}
            },
            "extract_time": 171120055.4,
            "source": "exchange rates api"
        }
    
        # Expected output DataFrame
        expected_df = pd.DataFrame([
            ("2024-02-20", "NZD", 1.062553),
            ("2024-02-20", "GBP", 0.519046),
            ("2024-02-21", "NZD", 1.060413),
            ("2024-02-21", "GBP", 0.518546)
        ], columns=['date', 'currency', 'rate'])
    
        # Call the function with sample data
        result_df = get_rates_dataframe(sample_data)
    
        # Check if the result matches the expected output
        self.assertTrue(expected_df.equals(result_df))

    def test_get_start_date(self):
        """
        Compares a generated date with the actual expected date from `get_start_date` function.
        
        - Setup: Sets up a test case where `date_threshold` is set to 30 days.  
        - Expected Result: Calculates the expected date by subtracting 30 days from the current date.
        - Function Execution: Calls the `get_start_date` function with the `date_threshold` value of 30 days.      
        - Assertion: Compares the result date returned by the `get_start_date` function with the expected date.
        """

        # Test case 2: date_threshold = 30
        date_threshold = 30
        expected_date = datetime.now().date() - timedelta(days=date_threshold)
        result_date = get_start_date(date_threshold)
        self.assertEqual(result_date, expected_date)

    def test_build_endpoint(self):
        """
        Compares a generated endpoint string with the actual expected string returned from `build_endpoint` function.
        
        - Setup: Sets up mock configuration values using the `ConfigParser` object to simulate the configuration file.    
        - Expected Result: Defines a mock function `mock_get_start_date` to simulate the behavior of the `get_start_date` function.     
        - Function Execution: Calls the `build_endpoint` function with the mock configuration.
        - Assertion: Compares the constructed endpoint returned by the `build_endpoint` function with the expected endpoint.
        """

        # Mock configuration values
        config = ConfigParser()
        config['arguments'] = {
            'endpoint': 'https://api.exchangeratesapi.io/v1/timeseries',
            'access_key': 'xxxx',
            'date_threshold': '30',
            'base_currency': 'AUD',
            'symbols': 'NZD'
        }

        # Mock get_start_date function
        def mock_get_start_date(date_threshold):
            return (datetime.now() - timedelta(days=date_threshold)).date()
        
        # Patching the get_start_date function
        with unittest.mock.patch('__main__.get_start_date', side_effect=mock_get_start_date):
            # Call the build_endpoint function with mock configuration
            endpoint = build_endpoint(config)
        
        # Assert that the constructed endpoint matches the expected value
        expected_endpoint = "https://api.exchangeratesapi.io/v1/timeseries?access_key=xxxx&start_date=2024-02-23&end_date=2024-03-24&base=AUD&symbols=NZD"
        self.assertEqual(endpoint, expected_endpoint)

In [20]:
unittest.main()

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<class 'SystemExit'>: False